In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [ ]:
import os
import pandas as pd
import json

# Retrieve data
___
Sourced from Golden Cheetah Open Data Project

In [ ]:
root_dir = 'E:\gc_opendata'
for a,b,c in os.walk(root_dir):
    athletes = b
    athletes.remove('INDEX')
    break

In [ ]:
target_athlete = athletes[0]
for a, b, c in os.walk(f'{root_dir}\{target_athlete}'):
    activities = c
    break

In [ ]:
target_activity = activities[-1]
with open(f'{root_dir}\{target_athlete}\{target_activity}', 'r') as f:
    ath_overview = f.read()

In [ ]:
slip = json.loads(ath_overview)

In [ ]:
slip = pd.DataFrame(pd.json_normalize(slip['RIDES']))

In [ ]:
slip

In [ ]:
activ_sum = pd.read_csv("C:/Users/ryand/git/TriHealth/activities.csv",parse_dates=['date'])

In [ ]:
activ_sum[['id','file']].groupby('id').count().sort_values('file').tail(30)

# Sample Explore

In [ ]:
target_id = "7567ac31-d479-4b25-9538-541ae8085288"

In [ ]:
act_samp = activ_sum[activ_sum['id'] == target_id].copy()

In [ ]:
act_samp.columns.tolist()

In [ ]:
act_samp['athlete_weight'] = act_samp['20m_critical_power']/act_samp['20m_peak_wpk']
act_samp['athlete_weight'] = act_samp['athlete_weight'].ffill().bfill()

In [ ]:
act_samp.set_index('date',inplace=True)

In [ ]:
act_samp[act_samp['sport'] == 'Bike']['20m_critical_power'].rolling(window=90, min_periods=1).max().plot();

In [ ]:
act_samp.plot(kind='scatter',x='workout_time',y='average_hr')

In [ ]:
def calc_vo2(row, max_hr, resting_hr):
    if row['sport'] == 'Bike':
        percent_vo2 = (row['average_hr'] - resting_hr)/(max_hr - resting_hr)
        vo2_estimated = (((row['average_power']/75)*1000)/row['athlete_weight']) / percent_vo2
        # vo2_estimated = row['VO2max_Detected'] ## overide with Garmin VO2 Estimation
    elif row['sport'] == 'Run':
        percent_vo2 = (row['average_hr'] - resting_hr)/(max_hr - resting_hr)
        vo2_estimated = (210/row['average_speed']) / percent_vo2
    else:
        vo2_estimated =  0
    return vo2_estimated

In [ ]:
act_samp['VO2'] = act_samp.apply(lambda row: calc_vo2(row, 179, 45), axis=1)

In [ ]:
act_samp

# Modeling

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
        self.fc3 = nn.Linear(2, 1)
         
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=1)
    
net = NN()
print(net)